In [26]:
!pip install alpha_vantage

In [27]:
import pandas as pd
import numpy as np
import datetime as dt
import time
from alpha_vantage.timeseries import TimeSeries

In [28]:
class ScriptData(object):
    
    #initializing constructor 
    def __init__(self):
        self.main_var = pd.DataFrame()
        self.data = {}
    ## get method for cllecting json data 
    def __getJson__(self):
        return self.data
    ## set method for setting json data 
    def __setJson__(self,newkey):
            self.data =newkey
    ## set method for overloading dataframes for diffrent scripts
    def __setDataFrame__(self,newkey):
        self.main_var = newkey
    
    ## get method for getting dataframes for diffrent scripts
    def __getDataFrame__(self):
        return self.main_var
        
        
    def fetch_intraday_data(self,script):
        ts = TimeSeries(key='PBUU95WNPEBUZYZI')
# Get json object with the intraday data and another with  the call's metadata
        data,meta = ts.get_intraday(script)
        self.__setJson__(data)
        print(type(self.__getJson__()))
    
  
        
        
        
    def convert_intraday_data(self,script):
        df = pd.DataFrame.from_dict(self.data)
    
        df= df.transpose()
        df.reset_index(level = 0, inplace=True)
        df.columns = ["timestamp","open","high","low","close","volume"]

        self.__setDataFrame__(df)
        
    
        
        

In [29]:
script_data= ScriptData()
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data.main_var.head()

<class 'dict'>


,timestamp,open,high,low,close,volume
0,2022-12-23 20:00:00,89.1100,89.1200,89.0500,89.0500,2135
1,2022-12-23 19:45:00,89.0900,89.0900,89.0900,89.0900,140
2,2022-12-23 19:30:00,89.1900,89.1900,89.1900,89.1900,1672
3,2022-12-23 19:15:00,89.0900,89.0900,89.0900,89.0900,101
4,2022-12-23 19:00:00,89.0500,89.0900,89.0500,89.0900,1861


In [30]:
###calculating moving average

def indicator1(df,timeperiod):
    temp= pd.DataFrame()
    temp["close"] = df["close"]
    temp["MA"] = temp["close"].rolling(timeperiod).mean()
    indicator_df = pd.DataFrame()
    indicator_df["timestamp"] = df["timestamp"]
    indicator_df["indicator"] = temp["MA"]
    return indicator_df

indicator1(script_data.main_var,5)

,timestamp,indicator
0,2022-12-23 20:00:00,NaN
1,2022-12-23 19:45:00,NaN
2,2022-12-23 19:30:00,NaN
3,2022-12-23 19:15:00,NaN
4,2022-12-23 19:00:00,89.10200
...,...,...
95,2022-12-22 12:00:00,86.99336
96,2022-12-22 11:45:00,87.08472
97,2022-12-22 11:30:00,87.17072
98,2022-12-22 11:15:00,87.26772


In [42]:
class Strategy:
    def __init__(self,script):
        self.script = script
    def get_script_data(self):
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
        
    def get_signal(self):
        signals = pd.DataFrame()
        signals["timestamp"] = script_data.main_var["timestamp"]
        close_data = script_data.main_var["close"]
        indicator_data = indicator1(script_data.main_var,5)["indicator"]
        temp = pd.DataFrame()
        temp["prev_indicator"] = indicator_data.shift(1)
        temp = temp.astype(float)
        
        
        
        
        #print(signals)
            
        return temp
        
        
strat = Strategy("GOOGL")
strat.get_script_data()
strat.get_signal()

<class 'dict'>


,prev_indicator
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
95,86.86236
96,86.99336
97,87.08472
98,87.17072
